In [1]:
import numpy as np
import pandas as pd

from money_parser import price_dec

In [2]:
def curr_to_float(v): return float(price_dec(v)) if v else v
def tf_to_10(v): return 1 if v == 't' else 0
def nulls_to_0(v): return float(0 if (pd.isnull(v) or v == '') else v)

converters_to_columns = [
    (curr_to_float, ['price', 'weekly_price', 'monthly_price']),
    (tf_to_10, ['instant_bookable', 'host_is_superhost']),
    (nulls_to_0, ['bedrooms', 'bathrooms'])
]

converters = {col: conv for conv, cols in converters_to_columns for col in cols}

old_listings_with_censustracts_df = pd.read_csv('data/old_listings_with_census.csv', index_col='id', converters=converters)
old_listings_to_crime = pd.read_csv('data/old_listings_with_crime_scores_granular.csv', index_col='id')

c:\program files\python37\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
old_listings_with_crime_df = old_listings_with_censustracts_df.join(old_listings_to_crime, how='inner')

In [8]:
census_tracts_df = pd.read_csv('data/nyc_census_tracts.csv')
census_tracts_df

,CensusTract,County,Borough,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,36005000100,Bronx,Bronx,7703,7133,570,29.9,6.1,60.9,0.2,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,36005000200,Bronx,Bronx,5403,2659,2744,75.8,2.3,16.0,0.0,...,2.9,0.0,0.0,43.0,2308,80.8,16.2,2.9,0.0,7.7
2,36005000400,Bronx,Bronx,5915,2896,3019,62.7,3.6,30.7,0.0,...,1.4,0.5,2.1,45.0,2675,71.7,25.3,2.5,0.6,9.5
3,36005001600,Bronx,Bronx,5879,2558,3321,65.1,1.6,32.4,0.0,...,8.6,1.6,1.7,38.8,2120,75.0,21.3,3.8,0.0,8.7
4,36005001900,Bronx,Bronx,2591,1206,1385,55.4,9.0,29.0,0.0,...,3.0,2.4,6.2,45.4,1083,76.8,15.5,7.7,0.0,19.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2162,36085030302,Richmond,Staten Island,6279,3093,3186,35.8,28.7,17.6,0.0,...,0.9,0.0,1.3,46.6,2668,74.4,22.5,3.2,0.0,6.2
2163,36085031901,Richmond,Staten Island,2550,953,1597,27.1,6.2,60.4,0.0,...,9.5,0.0,0.0,38.2,710,73.8,21.8,4.4,0.0,13.8
2164,36085031902,Richmond,Staten Island,4611,2043,2568,20.9,14.7,61.9,0.0,...,0.5,0.0,2.7,40.9,1308,84.7,12.6,2.7,0.0,9.4
2165,36085032300,Richmond,Staten Island,1131,597,534,45.5,24.0,29.7,0.0,...,4.9,0.0,0.0,52.5,534,65.0,31.1,3.9,0.0,10.7


In [9]:
old_listings_all_df = old_listings_with_crime_df.join(census_tracts_df.set_index('CensusTract'), 'census_tracts', how='inner')
old_listings_all_df['census_tracts'] = old_listings_all_df['census_tracts'].astype(np.int64)

In [14]:
census_block_loc_df = pd.read_csv('data/census_block_loc.csv')

In [22]:
census_block_loc_df['tract'] = census_block_loc_df['BlockCode'].astype(str).map(lambda v: v[:-4]).astype(np.int64)

In [27]:
census_tract_to_average_location = census_block_loc_df[['tract', 'Latitude', 'Longitude']].groupby('tract').mean()
census_tract_to_average_location.rename(columns={
    'Latitude': 'avg_tract_latitude', 
    'Longitude': 'avg_tract_longitude'
}, inplace=True)

In [31]:
old_listings_all_df = old_listings_all_df.join(census_tract_to_average_location, 'census_tracts', how='inner')

In [39]:
old_listings_all_df.columns.tolist()

['listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',
 'squar

In [32]:
old_listings_all_df.to_csv('data/old_listings_all.csv')